In [ ]:
! aws s3 cp segments_ns_ga_225_244  s3://monlam.ai.stt/stt_pecha_tools/ --recursive

In [ ]:
STT_NS_GA = 5
STT_NS_GB = 8
STT_NS_GG = 14
STT_NS_GH = 15


group = "ga"
group_id = STT_NS_GA
from_id = 225
to_id   = 244
target_directory = f"segments_ns_{group}_{from_id}_{to_id}"
last_db_id = 329725

In [ ]:
import os
def getTimeSpan(filename):
    filename = os.path.splitext(os.path.basename(filename))[0]
    try:
        if "_to_" in filename:
            start, end = filename.split("_to_")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)/1000
        else:
            start, end = filename.split("-")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)
    except Exception as err:
        print(f"filename is:'{filename}'. Could not parse to get time span.")
        return 0
    
getTimeSpan("STT_AB00094_0057_290646_to_295860.mp3")
getTimeSpan("STT_AB00094_0057_290646_to_295860")

In [ ]:
import os
from tqdm.auto import tqdm
from transformers import pipeline

generator = pipeline(model="openpecha/wav2vec2_run8")
rows = []

for root, _, files in os.walk(target_directory):
        for file in tqdm(files):
            source_file_path = os.path.join(root, file)
            opt = generator(f"/media/monlamai/SSD/GitHub/split/STT_NS/{target_directory}/{file}")
            inf = opt['text']
            rows.append([file, f"https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/{file}", inf , getTimeSpan(file)])

In [ ]:
rows[0]

In [ ]:
import pandas as pd
df = pd.DataFrame(rows, columns =['file_name', 'url', 'inference_transcript', 'audio_duration'])

In [ ]:
df[['inference_transcript','url']].iloc[0:10].to_dict()

In [ ]:
df['group_id'] = group_id
df['state'] = 'imported'

In [ ]:
df['id'] = df.index + last_db_id + 1

In [ ]:
df.head()

In [ ]:
df.to_csv(f"segments_ns_{group}.csv", index=False)

In [ ]:
df[df['inference_transcript'].apply(lambda x: len(x) < 500)].shape

In [ ]:
df.shape